In [2]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# Selenium WebDriver 설정
driver = wb.Chrome()
driver.maximize_window()

# 페이지 로드 타임아웃 시간 설정 (예: 60초)
driver.set_page_load_timeout(20)

# 기본 URL
base_url = 'https://gall.dcinside.com/mgallery/board/lists/?id=hikikomori&page='

def fetch_filtered_elements(page_number):
    url = base_url + str(page_number)
    
    try:
        driver.get(url)
        time.sleep(2)  # 페이지가 로드될 때까지 대기
        
        # 모든 게시글 제목 요소 가져오기
        all_elements = driver.find_elements(By.CSS_SELECTOR, '#container > section.left_content > article:nth-child(3) > div.gall_listwrap.list > table > tbody > tr > td.gall_tit.ub-word > a:nth-child(1)')
        
        filtered_posts = []
        
        for element in all_elements:
            parent_row = element.find_element(By.XPATH, './ancestor::tr')
            subject_td = parent_row.find_element(By.CSS_SELECTOR, 'td.gall_subject')
            
            if subject_td.text.strip() == '💬':  # 특정 주제 필터링
                title = element.text.strip()
                link = element.get_attribute('href')
                filtered_posts.append({'title': title, 'link': link})
        
        return filtered_posts
    except Exception as e:
        print(f"Error fetching filtered elements on page {page_number}: {e}")
        return []

def fetch_post_details(link):
    try:
        driver.get(link)
        
        # 특정 요소가 로드될 때까지 대기 (최대 30초)
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '#container > section > article:nth-child(3) > div.view_content_wrap > header > div > h3 > span.title_subject'))
        )

        # 제목 요소 가져오기
        title_element = driver.find_element(By.CSS_SELECTOR, '#container > section > article:nth-child(3) > div.view_content_wrap > header > div > h3 > span.title_subject')
        title = title_element.text.strip()

        # 내용 요소 가져오기
        content_element = driver.find_element(By.CSS_SELECTOR, '#container > section > article:nth-child(3) > div.view_content_wrap > div > div.inner.clear > div.writing_view_box > div.write_div')
        content = content_element.text.strip()

        return {'title': title, 'content': content}
    except Exception as e:
        print(f"Error fetching post details: {e}")
        return None

def save_data(post_details_list, filename='post_details.csv'):
    df = pd.DataFrame(post_details_list)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"Data has been saved to {filename}")

def main():
    post_details_list = []

    try:
        for page_number in range(1, 101):  # 1페이지부터 30페이지까지
            print(f"Fetching page {page_number}...")
            posts = fetch_filtered_elements(page_number)
            
            if posts:
                for post in posts:
                    print(f"Fetching details for post: {post['title']}")
                    post_details = fetch_post_details(post['link'])
                    if post_details:
                        post_details_list.append(post_details)
                        print(f"제목: {post_details['title']}")
                        print(f"내용: {post_details['content']}")
                        print('-' * 50)
            else:
                print(f"No posts found on page {page_number}")
    
    except Exception as e:
        print(f"An error occurred: {e}")
    
    finally:
        if post_details_list:
            save_data(post_details_list)
        driver.quit()  # WebDriver 종료

if __name__ == "__main__":
    main()


Fetching page 1...
Fetching details for post: 죽을때까지 못 해본 사람 잇을까?
제목: 죽을때까지 못 해본 사람 잇을까?
내용: ㅇ
--------------------------------------------------
Fetching details for post: 나도 해보고 싶다
제목: 나도 해보고 싶다
내용: ㅇ
--------------------------------------------------
Fetching details for post: 이거보는중
제목: 이거보는중
내용: https://youtu.be/E6-vRk_64YU?si=r4Bi0Wi37Qd0GxVq
--------------------------------------------------
Fetching details for post: 걍 인생 좆같네 ㅈㄴ 안풀린다
제목: 걍 인생 좆같네 ㅈㄴ 안풀린다
내용: 하 ㅅㅂ
--------------------------------------------------
Fetching details for post: 나도이제슬슬집나가라고한다 념글처럼
제목: 나도이제슬슬집나가라고한다 념글처럼
내용: 26년에 20중반이면 나보다어리긴하네
그냥 흙수저의 운명인듯
--------------------------------------------------
Fetching details for post: 이거보는중
제목: 이거보는중
내용: https://youtu.be/5R-L0SPUUIo?si=E090bCulYs135NGc
--------------------------------------------------
Fetching details for post: 인생망해서 우울하네
제목: 인생망해서 우울하네
내용: 살기가싫다
--------------------------------------------------
Fetching details for post: 가슴을 데인 것처럼 눈물에 베인것처럼
제목: 가슴을 